# Problem 1: Binary Words and Operations

## Introduction

This problem implements seven core bitwise functions used in the SHA-256 cryptographic hash algorithm, as defined in the FIPS 180-4 Secure Hash Standard. SHA-256 is widely used in security applications like SSL/TLS, Bitcoin, and digital signatures. These functions perform bitwise operations on 32-bit words to create the confusion and diffusion properties essential for cryptographic security. The logical functions (Ch, Maj, Parity) operate on individual bits, while the rotation functions (Sigma and sigma) combine rotations and shifts with XOR operations to ensure each input bit affects many output bits. Together, they make it computationally infeasible to reverse the hash or find collisions.

## Solution Approach

The solution uses Python's numpy library to ensure all operations are performed on 32-bit unsigned integers (np.uint32), as required by the SHA-256 specification. Each function follows the exact mathematical formulas from FIPS 180-4: the logical functions use AND (&), OR (|), XOR (^), and NOT (~) operators, while the rotation functions use helper functions ROTR (circular right rotation) and SHR (logical right shift). The ROTR operation is implemented using bit shifts and OR operations to wrap bits from the right end to the left, while SHR simply shifts bits right and fills with zeros. This approach ensures compatibility with the standard and produces verifiable results against known test vectors.

## Functions to Implement:

1. `Parity(x, y, z)` - XOR operation across three values
2. `Ch(x, y, z)` - Choice function (conditional bit selection)
3. `Maj(x, y, z)` - Majority function (returns majority bit value)
4. `Sigma0(x)` - written as Σ₀²⁵⁶(x) in the standard
5. `Sigma1(x)` - written as Σ₁²⁵⁶(x) in the standard
6. `sigma0(x)` - written as σ₀²⁵⁶(x) in the standard
7. `sigma1(x)` - written as σ₁²⁵⁶(x) in the standard

## References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 4.1.2, Page 10 - SHA-256 Functions

In [17]:
# ============================================================================
# IMPORTS - All package imports for this notebook
# ============================================================================
import numpy as np

In [18]:
# ----------------------------------------------------------------------------
# PART 1: Parity Function
# ----------------------------------------------------------------------------

def Parity(x, y, z):
    """

    Parity function from SHA-256 specification.
    
    Returns the bitwise XOR of x, y, and z.
    Formula: x ⊕ y ⊕ z
    
    This function is used in the SHA-1 algorithm but included here
    for completeness of bitwise operations.
     
    Args:
        x, y, z: 32-bit unsigned integers
    Returns:
        np.uint32: Result of x XOR y XOR z
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Perform XOR operation on all three values
    return np.uint32(x ^ y ^ z)  
# ----------------------------------------------------------------------------
# PART 2: Choice (Ch) Function
# ----------------------------------------------------------------------------


def Ch(x, y, z):
    """
    Choice function from SHA-256 specification.
    
    For each bit position, chooses bit from y if x is 1,
    otherwise chooses bit from z.
    
    Formula: (x ∧ y) ⊕ (¬x ∧ z)
    
    Used in: SHA-256 compression function (Step 1 of each round)
    
    Args:
        x, y, z: 32-bit unsigned integers
    
    Returns:
        np.uint32: Result of the choice operation
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Where x is 1, choose y; where x is 0, choose z
    return np.uint32((x & y) ^ (~x & z))
# ----------------------------------------------------------------------------
# PART 3: Majority (Maj) Function
# ----------------------------------------------------------------------------

def Maj(x, y , z):
    """
     Majority function from SHA-256 specification.
    
    For each bit position, returns 1 if at least two of the
    three corresponding bits in x, y, z are 1.
    
    Formula: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Used in: SHA-256 compression function (Step 2 of each round)
    
    Args:
        x, y, z: 32-bit unsigned integers
    
    Returns:
        np.uint32: Result of the majority operation
    """
    # Convert all inputs to 32-bit unsigned integers
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)
    # Return 1 where at least 2 of the 3 bits are 1
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

# ============================================================================
# HELPER FUNCTIONS: ROTR (Rotate Right) and SHR (Shift Right)
# ============================================================================

def ROTR(n, x):
    """
    Rotate right (circular right shift) operation.
    
    Bits shifted off the right end wrap around to the left.
    
    Args:
        n: Number of positions to rotate
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: x rotated right by n positions
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))

def SHR(n, x):
    """
    Shift right (logical right shift) operation.
    
    Bits shifted off the right are lost, zeros fill from the left.
    
    Args:
        n: Number of positions to shift
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: x shifted right by n positions
    """
    x = np.uint32(x)
    return np.uint32(x >> n )

# ----------------------------------------------------------------------------
# PART 4: Sigma0 Function (Uppercase Σ₀²⁵⁶)
# ----------------------------------------------------------------------------
def Sigma0(x):
    """
    Sigma0 function from SHA-256 specification.
    
    Combines three rotations of x with XOR operation.
    Formula: ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Used in: SHA-256 compression function for computing temp2 value
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of Sigma0 operation
    """
    x = np.uint32(x)
    # XOR of three different rotations
    return np.uint32(ROTR(2, x) ^ ROTR(13, x) ^ ROTR(22, x))

# ----------------------------------------------------------------------------
# PART 5: Sigma1 Function (Uppercase Σ₁²⁵⁶)
# ----------------------------------------------------------------------------

def Sigma1(x):
    """

    Sigma1 function from SHA-256 specification.
    Formula :  ROTR^6(x) XOR ROTR^11(x) XOR ROTR^25(x)

    Used in: SHA-256 compression function for computing temp1 value
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of Sigma1 operation
    """
    x = np.uint32(x)
    # XOR of three different rotations
    return np.uint32(ROTR(6, x) ^ ROTR(11, x) ^ ROTR(25, x))


# ----------------------------------------------------------------------------
# PART 6: sigma0 Function (Lowercase σ₀²⁵⁶)
# ----------------------------------------------------------------------------
def sigma0(x):
    """
    sigma0 (lowercase) function from SHA-256 specification.
    
    Combines two rotations and one shift with XOR operation.
    Formula: ROTR^7(x) XOR ROTR^18(x) XOR SHR^3(x)
    
    Note: Uses SHR (shift) for the third operation, not ROTR (rotate)
    
    Used in: SHA-256 message schedule expansion (extending 16 words to 64 words)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of sigma0 operation
    """
    x = np.uint32(x)
    # XOR of two rotations and one shift
    return np.uint32(ROTR(7, x) ^ ROTR(18, x) ^ SHR(3, x))

# ----------------------------------------------------------------------------
# PART 7: sigma1 Function (Lowercase σ₁²⁵⁶)
# ----------------------------------------------------------------------------
def sigma1(x):
    """
    sigma1 (lowercase) function from SHA-256 specification.
    
    Combines two rotations and one shift with XOR operation.
    Formula: ROTR^17(x) XOR ROTR^19(x) XOR SHR^10(x)
    
    Note: Uses SHR (shift) for the third operation, not ROTR (rotate)
    
    Used in: SHA-256 message schedule expansion (extending 16 words to 64 words)
    
    Args:
        x: 32-bit unsigned integer
    
    Returns:
        np.uint32: Result of sigma1 operation
    """
    x = np.uint32(x)
    # XOR of two rotations and one shift
    return np.uint32(ROTR(17, x) ^ ROTR(19, x) ^ SHR(10, x))



In [19]:
# Test 1 the Parity function
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)

result = Parity(x, y, z)
print(f"Parity({x}, {y}, {z}) = {result}")
print(f"Binary: {bin(result)}")


#Test 1.2: All zeros
x, y, z = np.uint32(0), np.uint32(0), np.uint32(0)
result = Parity(x, y, z)
print(f"Test 1.2: Parity(0, 0, 0) = {result} (Expected: 0)")

# Test 1.3: All ones
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0xFFFFFFFF)
z = np.uint32(0xFFFFFFFF)
result = Parity(x, y, z)
print(f"Test 1.3: Parity(all 1s) = {hex(result)} (Expected: 0xffffffff)")



Parity(12, 10, 6) = 0
Binary: 0b0
Test 1.2: Parity(0, 0, 0) = 0 (Expected: 0)
Test 1.3: Parity(all 1s) = 0xffffffff (Expected: 0xffffffff)


In [20]:
# 2.Test the Ch function
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)

result = Ch(x, y, z)
print(f"Ch({x}, {y}, {z}) = {result}")
print(f"Binary: {bin(result)}")

# Test 2.2 : x all ones (chooses y)
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0x12345678)
z = np.uint32(0x9ABCDEF0)
result = Ch(x, y, z)
print(f"Test 2.2: Ch(all 1s, y, z) = {hex(result)} (Expected: 0x12345678)")

# Test 2.3: x all zeros (chooses z)
x = np.uint32(0)
y = np.uint32(0x12345678)
z = np.uint32(0x9ABCDEF0)
result = Ch(x, y, z)
print(f"Test 2.3: Ch(all 0s, y, z) = {hex(result)} (Expected: 0x9abcdef0)")

print("\n" + "=" * 70)




Ch(12, 10, 6) = 10
Binary: 0b1010
Test 2.2: Ch(all 1s, y, z) = 0x12345678 (Expected: 0x12345678)
Test 2.3: Ch(all 0s, y, z) = 0x9abcdef0 (Expected: 0x9abcdef0)



In [21]:
# ----------------------------------------------------------------------------
# PART 3: Maj (Majority) Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 3: Maj (Majority) Function Tests ---")

# Test 3.1: Basic test
x = np.uint32(0b1100)
y = np.uint32(0b1010)
z = np.uint32(0b0110)
result = Maj(x, y, z)
print(f"Test 3.1: Maj({x}, {y}, {z}) = {result}")
print(f"         Binary: {bin(result)}")

# Test 3.2: All zeros
x, y, z = np.uint32(0), np.uint32(0), np.uint32(0)
result = Maj(x, y, z)
print(f"Test 3.2: Maj(0, 0, 0) = {result} (Expected: 0)")

# Test 3.3: All ones
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0xFFFFFFFF)
z = np.uint32(0xFFFFFFFF)
result = Maj(x, y, z)
print(f"Test 3.3: Maj(all 1s) = {hex(result)} (Expected: 0xffffffff)")

# Test 3.4: Two agree on 1s (majority wins)
x = np.uint32(0xFFFFFFFF)
y = np.uint32(0xFFFFFFFF)
z = np.uint32(0x00000000)
result = Maj(x, y, z)
print(f"Test 3.4: Maj(1s, 1s, 0s) = {hex(result)} (Expected: 0xffffffff)")

print("\n" + "=" * 70)



--- PART 3: Maj (Majority) Function Tests ---
Test 3.1: Maj(12, 10, 6) = 14
         Binary: 0b1110
Test 3.2: Maj(0, 0, 0) = 0 (Expected: 0)
Test 3.3: Maj(all 1s) = 0xffffffff (Expected: 0xffffffff)
Test 3.4: Maj(1s, 1s, 0s) = 0xffffffff (Expected: 0xffffffff)



In [22]:
# ----------------------------------------------------------------------------
# PART 4: Sigma0 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 4: Sigma0 (Uppercase Σ₀) Function Tests ---")

# Test 4.1: Basic test
x = np.uint32(0x12345678)
result = Sigma0(x)
print(f"Test 4.1: Sigma0(0x12345678) = {hex(result)}")

# Test 4.2: All zeros
x = np.uint32(0)
result = Sigma0(x)
print(f"Test 4.2: Sigma0(0) = {result} (Expected: 0)")

# Test 4.3: All ones
x = np.uint32(0xFFFFFFFF)
result = Sigma0(x)
print(f"Test 4.3: Sigma0(all 1s) = {hex(result)} (Expected: 0xffffffff)")

# Test 4.4: Alternating bits
x = np.uint32(0xAAAAAAAA)
result = Sigma0(x)
print(f"Test 4.4: Sigma0(0xAAAAAAAA) = {hex(result)}")


--- PART 4: Sigma0 (Uppercase Σ₀) Function Tests ---
Test 4.1: Sigma0(0x12345678) = 0x66146474
Test 4.2: Sigma0(0) = 0 (Expected: 0)
Test 4.3: Sigma0(all 1s) = 0xffffffff (Expected: 0xffffffff)
Test 4.4: Sigma0(0xAAAAAAAA) = 0x55555555


In [23]:
# ----------------------------------------------------------------------------
# PART 5: Sigma1 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 5: Sigma1 (Uppercase Σ₁) Function Tests ---")

# Test 5.1: Basic test
x = np.uint32(0x12345678)
result = Sigma1(x)
print(f"Test 5.1: Sigma1(0x12345678) = {hex(result)}")

# Test 5.2: All zeros
x = np.uint32(0)
result = Sigma1(x)
print(f"Test 5.2: Sigma1(0) = {result} (Expected: 0)")

# Test 5.3: All ones
x = np.uint32(0xFFFFFFFF)
result = Sigma1(x)
print(f"Test 5.3: Sigma1(all 1s) = {hex(result)} (Expected: 0xffffffff)")

# Test 5.4: Alternating bits
x = np.uint32(0xAAAAAAAA)
result = Sigma1(x)
print(f"Test 5.4: Sigma1(0xAAAAAAAA) = {hex(result)}")


--- PART 5: Sigma1 (Uppercase Σ₁) Function Tests ---
Test 5.1: Sigma1(0x12345678) = 0x3561abda
Test 5.2: Sigma1(0) = 0 (Expected: 0)
Test 5.3: Sigma1(all 1s) = 0xffffffff (Expected: 0xffffffff)
Test 5.4: Sigma1(0xAAAAAAAA) = 0xaaaaaaaa


In [24]:
# ----------------------------------------------------------------------------
# PART 6: sigma0 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 6: sigma0 (Lowercase σ₀) Function Tests ---")

# Test 6.1: Basic test
x = np.uint32(0x12345678)
result = sigma0(x)
print(f"Test 6.1: sigma0(0x12345678) = {hex(result)}")

# Test 6.2: All zeros
x = np.uint32(0)
result = sigma0(x)
print(f"Test 6.2: sigma0(0) = {result} (Expected: 0)")

# Test 6.3: All ones
x = np.uint32(0xFFFFFFFF)
result = sigma0(x)
print(f"Test 6.3: sigma0(all 1s) = {hex(result)}")

# Test 6.4: Power of 2
x = np.uint32(0x80000000)
result = sigma0(x)
print(f"Test 6.4: sigma0(0x80000000) = {hex(result)}")


--- PART 6: sigma0 (Lowercase σ₀) Function Tests ---
Test 6.1: sigma0(0x12345678) = 0xe7fce6ee
Test 6.2: sigma0(0) = 0 (Expected: 0)
Test 6.3: sigma0(all 1s) = 0x1fffffff
Test 6.4: sigma0(0x80000000) = 0x11002000


In [25]:
# ----------------------------------------------------------------------------
# PART 7: sigma1 Function Tests
# ----------------------------------------------------------------------------
print("\n--- PART 7: sigma1 (Lowercase σ₁) Function Tests ---")

# Test 7.1: Basic test
x = np.uint32(0x12345678)
result = sigma1(x)
print(f"Test 7.1: sigma1(0x12345678) = {hex(result)}")

# Test 7.2: All zeros
x = np.uint32(0)
result = sigma1(x)
print(f"Test 7.2: sigma1(0) = {result} (Expected: 0)")

# Test 7.3: All ones
x = np.uint32(0xFFFFFFFF)
result = sigma1(x)
print(f"Test 7.3: sigma1(all 1s) = {hex(result)}")

# Test 7.4: Power of 2
x = np.uint32(0x80000000)
result = sigma1(x)
print(f"Test 7.4: sigma1(0x80000000) = {hex(result)}")

print("\n" + "=" * 70)
print("ALL PROBLEM 1 FUNCTIONS COMPLETE!")
print("=" * 70)


--- PART 7: sigma1 (Lowercase σ₁) Function Tests ---
Test 7.1: sigma1(0x12345678) = 0xa1f78649
Test 7.2: sigma1(0) = 0 (Expected: 0)
Test 7.3: sigma1(all 1s) = 0x3fffff
Test 7.4: sigma1(0x80000000) = 0x205000

ALL PROBLEM 1 FUNCTIONS COMPLETE!


## Conclusion

This problem successfully implemented all seven bitwise functions required for the SHA-256 algorithm as specified in FIPS 180-4. The logical functions (Parity, Ch, Maj) provide non-linear bit mixing through conditional operations and majority voting, while the rotation functions (Sigma0, Sigma1, sigma0, sigma1) create diffusion by combining multiple rotations and shifts with XOR operations. All functions were tested with multiple test cases including edge cases (all zeros, all ones, and specific bit patterns) to verify correctness against the specification. The implementation uses numpy's uint32 type to ensure proper 32-bit arithmetic and modulo 2^32 behavior. These seven functions form the cryptographic foundation for the complete SHA-256 hash algorithm that will be implemented in subsequent problems.

## Problem 2: Fractional Parts of Cube Roots

### Introduction

This problem calculates the constants used in the SHA-256 message schedule, specifically the round constants K. These constants are derived from the first 32 bits of the fractional parts of the cube roots of the first 64 prime numbers. Using mathematical constants like these ensures that the values are "nothing up my sleeve" numbers - they're derived from a transparent mathematical process rather than arbitrary choices, which is important for cryptographic security and trust in the algorithm.

### Solution Approach

The solution implements a prime number generator using trial division, calculates cube roots using numpy's cbrt function, extracts the fractional part by subtracting the integer part, and then converts the fractional part to a 32-bit hexadecimal representation by multiplying by 2^32. The results are verified against the constants listed in FIPS 180-4 (page 11) to ensure correctness.

### References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 4.2.2, Page 11 - SHA-256 Constants
- Relevance: Official specification listing the 64 round constants derived from cube roots of the first 64 primes.

**Wikipedia - Nothing-up-my-sleeve number**
- URL: https://en.wikipedia.org/wiki/Nothing-up-my-sleeve_number
- Relevance: Explains why cryptographic algorithms use mathematically derived constants rather than arbitrary values to prevent hidden backdoors and increase trust.

**Wikipedia - Prime number**
- URL: https://en.wikipedia.org/wiki/Prime_number
- Relevance: Provides background on prime number properties and generation algorithms used in this implementation.

In [26]:
# ============================================================================
# PROBLEM 2: Fractional Parts of Cube Roots
# ============================================================================



# ----------------------------------------------------------------------------
# STEP 1: Prime Number Generator
# ----------------------------------------------------------------------------

def primes(n):
    """
    Generate the first n prime numbers.
    
    Uses trial division method: checks if each candidate number
    is divisible by any previously found primes.
    
    Args:
        n: Number of primes to generate
    
    Returns:
        list: List of the first n prime numbers
    
    Example:
        >>> primes(5)
        [2, 3, 5, 7, 11]
    """
    if n <= 0:
        return []
    
    prime_list = []  # Store found primes
    candidate = 2    # Start checking from 2 (first prime)
    
    while len(prime_list) < n:
        is_prime = True
        
        # Check if candidate is divisible by any existing prime
        for prime in prime_list:
            if candidate % prime == 0:
                is_prime = False
                break
            # Optimization: only check up to sqrt(candidate)
            if prime * prime > candidate:
                break
        
        if is_prime:
            prime_list.append(candidate)
        
        candidate += 1
    
    return prime_list


# ----------------------------------------------------------------------------
# Test the Prime Generator
# ----------------------------------------------------------------------------
print("=" * 70)
print("PROBLEM 2: Testing Prime Number Generator")
print("=" * 70)

# Test with first 10 primes
first_10 = primes(10)
print(f"\nFirst 10 primes: {first_10}")
print(f"Expected: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]")

# Test with first 20 primes
first_20 = primes(20)
print(f"\nFirst 20 primes: {first_20}")

# Generate the 64 primes we need for SHA-256
first_64 = primes(64)
print(f"\nFirst 64 primes generated successfully!")
print(f"First prime: {first_64[0]}, Last prime: {first_64[-1]}")

print("=" * 70)

# ----------------------------------------------------------------------------
# STEP 2: Calculate Cube Roots and Extract Fractional Parts
# ----------------------------------------------------------------------------

# Get the first 64 primes
primes_64 = primes(64)

print("\n" + "=" * 70)
print("Calculating Cube Roots of First 64 Primes")
print("=" * 70)

# Calculate cube roots and extract fractional parts
cube_roots = []
fractional_parts = []

for i, prime in enumerate(primes_64):
    # Calculate cube root
    cbrt = np.cbrt(prime)
    
    # Extract fractional part (remove integer part)
    frac = cbrt - int(cbrt)
    
    cube_roots.append(cbrt)
    fractional_parts.append(frac)
    
    # Display first 5 as examples
    if i < 5:
        print(f"Prime {prime:3d}: cbrt = {cbrt:.10f}, fractional part = {frac:.10f}")

print(f"\n... (calculated for all 64 primes)")
print(f"Total fractional parts calculated: {len(fractional_parts)}")
print("=" * 70)

# ----------------------------------------------------------------------------
# STEP 3: Convert Fractional Parts to 32-bit Hexadecimal
# ----------------------------------------------------------------------------

print("\n" + "=" * 70)
print("Converting Fractional Parts to 32-bit Hexadecimal Constants")
print("=" * 70)

# Convert fractional parts to 32-bit hex values
hex_constants = []

for i, frac in enumerate(fractional_parts):
    # Multiply by 2^32 to get the first 32 bits
    # Convert to unsigned 32-bit integer
    value = np.uint32(frac * (2**32))
    hex_constants.append(value)
    
    # Display first 10 as examples
    if i < 10:
        print(f"K[{i:2d}] = 0x{value:08x}  (prime = {primes_64[i]:3d})")

print(f"\n... (calculated for all 64 primes)")
print(f"\nTotal constants generated: {len(hex_constants)}")

# Display all 64 constants in a formatted table
print("\n" + "=" * 70)
print("All 64 SHA-256 Round Constants (K)")
print("=" * 70)

for i in range(0, 64, 4):
    # Print 4 constants per line
    line = "  ".join([f"0x{hex_constants[j]:08x}" for j in range(i, min(i+4, 64))])
    print(f"K[{i:2d}-{min(i+3, 63):2d}]: {line}")

print("=" * 70)


# ----------------------------------------------------------------------------
# STEP 4: Verify Against FIPS 180-4 Standard (Page 11)
# ----------------------------------------------------------------------------

print("\n" + "=" * 70)
print("Verification Against FIPS 180-4 Standard")
print("=" * 70)

# Official K constants from FIPS 180-4, page 11
official_K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# Compare our calculated values with official values
all_match = True
mismatches = []

for i in range(64):
    if hex_constants[i] != official_K[i]:
        all_match = False
        mismatches.append(i)

if all_match:
    print("\n✓ SUCCESS! All 64 constants match FIPS 180-4 specification!")
    print(f"\nVerification: {len(hex_constants)} calculated = {len(official_K)} official")
else:
    print(f"\n✗ MISMATCH! {len(mismatches)} constants do not match:")
    for i in mismatches:
        print(f"  K[{i}]: calculated = 0x{hex_constants[i]:08x}, "
              f"official = 0x{official_K[i]:08x}")

# Display a few examples of matching values
print("\nSample verification (first 5 constants):")
for i in range(5):
    match_str = "✓" if hex_constants[i] == official_K[i] else "✗"
    print(f"{match_str} K[{i}]: calculated = 0x{hex_constants[i]:08x}, "
          f"official = 0x{official_K[i]:08x}")

print("=" * 70)



PROBLEM 2: Testing Prime Number Generator

First 10 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Expected: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

First 20 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

First 64 primes generated successfully!
First prime: 2, Last prime: 311

Calculating Cube Roots of First 64 Primes
Prime   2: cbrt = 1.2599210499, fractional part = 0.2599210499
Prime   3: cbrt = 1.4422495703, fractional part = 0.4422495703
Prime   5: cbrt = 1.7099759467, fractional part = 0.7099759467
Prime   7: cbrt = 1.9129311828, fractional part = 0.9129311828
Prime  11: cbrt = 2.2239800906, fractional part = 0.2239800906

... (calculated for all 64 primes)
Total fractional parts calculated: 64

Converting Fractional Parts to 32-bit Hexadecimal Constants
K[ 0] = 0x428a2f98  (prime =   2)
K[ 1] = 0x71374491  (prime =   3)
K[ 2] = 0xb5c0fbcf  (prime =   5)
K[ 3] = 0xe9b5dba5  (prime =   7)
K[ 4] = 0x3956c25b  (prime =  11)
K[ 5] = 0x59f111f1  (prim

### Conclusion

This problem successfully generated all 64 round constants (K) used in the SHA-256 algorithm by calculating the first 32 bits of the fractional parts of cube roots of the first 64 prime numbers. The implementation used a trial division algorithm to generate primes, numpy's cbrt function for cube root calculations, and bit manipulation to extract the 32-bit fractional representations. All 64 calculated constants were verified to exactly match the official values specified in FIPS 180-4 (page 11), confirming the correctness of the implementation. These constants, derived from a transparent mathematical process, exemplify the "nothing up my sleeve" principle in cryptographic design, ensuring no hidden weaknesses or backdoors in the algorithm.

## Problem 3: Padding

### Introduction

This problem implements the message padding and parsing mechanism for SHA-256 as specified in FIPS 180-4, Sections 5.1.1 and 5.2.1. SHA-256 requires that messages be padded to a multiple of 512 bits before processing. The padding ensures that messages of any length can be processed in uniform 512-bit blocks. The padding scheme appends a '1' bit, followed by zero or more '0' bits, and concludes with a 64-bit representation of the original message length. This standardized padding is crucial for the security of the hash function, as it prevents length extension attacks and ensures that different messages cannot produce the same padded output.

### Solution Approach

The solution implements a Python generator function that yields 512-bit (64-byte) blocks from a message. The function first calculates the required padding: append a byte with value 0x80 (binary 10000000), then add enough zero bytes so that the message length plus padding is 64 bytes short of a multiple of 512 bits (64 bytes). Finally, append the original message length as a 64-bit big-endian integer. The generator yields each complete 512-bit block as a bytes object, making it memory-efficient for processing large messages.

### References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 5.1.1, Page 13 - SHA-256 Padding
- Section 5.2.1, Page 14 - SHA-256 Parsing
- Relevance: Official specification for message padding and block parsing required before hash computation.

**Wikipedia - Padding (cryptography)**
- URL: https://en.wikipedia.org/wiki/Padding_(cryptography)
- Relevance: Explains why padding is necessary in cryptographic hash functions and describes different padding schemes including Merkle-Damgård construction used in SHA-256.

**Python Documentation - Generators**
- URL: https://docs.python.org/3/howto/functional.html#generators
- Relevance: Explains Python generator functions and the yield statement used for memory-efficient block processing.

In [27]:
# ============================================================================
# PROBLEM 3: Padding
# ============================================================================

def block_parse(msg):
    """
    Generator function that pads and parses a message into 512-bit blocks.
    
    Implements SHA-256 padding from FIPS 180-4:
    - Append bit '1' (byte 0x80)
    - Append zero bytes until length ≡ 448 (mod 512) bits
    - Append 64-bit big-endian representation of original message length
    
    Args:
        msg: bytes object containing the message to be padded and parsed
    
    Yields:
        bytes: 512-bit (64-byte) blocks of the padded message
    
    Example:
        >>> for block in block_parse(b"abc"):
        ...     print(len(block))  # Should print 64
        64
    """
    # Get the original message length in bits
    msg_len_bits = len(msg) * 8
    
    # Start with the original message
    padded = bytearray(msg)
    
    # Step 1: Append the '1' bit (0x80 = 10000000 in binary)
    padded.append(0x80)
    
    # Step 2: Calculate how many zero bytes we need
    # We need the total length to be 448 bits (56 bytes) mod 512 bits (64 bytes)
    # Length so far: len(msg) + 1 byte (the 0x80)
    # We need: (current_length + zero_bytes + 8) % 64 == 0
    # Or equivalently: current_length + zero_bytes ≡ 56 (mod 64)
    
    current_len = len(padded)
    
    # Calculate zero bytes needed
    if current_len % 64 <= 56:
        # We can fit in the current block
        zero_bytes = 56 - (current_len % 64)
    else:
        # We need to go into the next block
        zero_bytes = 64 - (current_len % 64) + 56
    
    # Append the zero bytes
    padded.extend(b'\x00' * zero_bytes)
    
    # Step 3: Append the original message length as 64-bit big-endian integer
    padded.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    # Step 4: Yield 512-bit (64-byte) blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

### Testing the Padding Function

The `block_parse()` function is tested with five different scenarios to verify correctness and handle edge cases:

1. **Empty message** - Tests the minimum case where only padding is added
2. **Short message ("abc")** - Standard test case from FIPS 180-4 examples
3. **55-byte message** - Boundary case that fits exactly in one block with padding
4. **56-byte message** - Boundary case that requires two blocks (padding won't fit in first block)
5. **120-byte message** - Tests multiple block generation

Each test verifies that:
- All blocks are exactly 64 bytes (512 bits)
- The correct number of blocks is generated
- The padding format matches SHA-256 specification

In [28]:
# ----------------------------------------------------------------------------
# Test the block_parse generator
# ----------------------------------------------------------------------------
print("=" * 70)
print("PROBLEM 3: Testing Message Padding and Parsing")
print("=" * 70)

# Test 1: Empty message
print("\n--- Test 1: Empty message ---")
msg = b""
blocks = list(block_parse(msg))
print(f"Message: {repr(msg)}")
print(f"Message length: {len(msg)} bytes")
print(f"Number of blocks: {len(blocks)}")
print(f"First block length: {len(blocks[0])} bytes (expected: 64)")
print(f"First block (hex): {blocks[0].hex()}")

# Test 2: Short message - "abc"
print("\n--- Test 2: Short message 'abc' ---")
msg = b"abc"
blocks = list(block_parse(msg))
print(f"Message: {repr(msg)}")
print(f"Message length: {len(msg)} bytes")
print(f"Number of blocks: {len(blocks)}")
print(f"First block length: {len(blocks[0])} bytes (expected: 64)")
print(f"First block (hex): {blocks[0].hex()}")

# Test 3: Message exactly 55 bytes (boundary case - fits in one block)
print("\n--- Test 3: 55-byte message (boundary case) ---")
msg = b"a" * 55
blocks = list(block_parse(msg))
print(f"Message: 55 'a' characters")
print(f"Message length: {len(msg)} bytes")
print(f"Number of blocks: {len(blocks)}")
for i, block in enumerate(blocks):
    print(f"Block {i+1} length: {len(block)} bytes")

# Test 4: Message exactly 56 bytes (boundary case - needs two blocks)
print("\n--- Test 4: 56-byte message (boundary case) ---")
msg = b"a" * 56
blocks = list(block_parse(msg))
print(f"Message: 56 'a' characters")
print(f"Message length: {len(msg)} bytes")
print(f"Number of blocks: {len(blocks)}")
for i, block in enumerate(blocks):
    print(f"Block {i+1} length: {len(block)} bytes")

# Test 5: Long message (multiple blocks)
print("\n--- Test 5: 120-byte message (multiple blocks) ---")
msg = b"x" * 120
blocks = list(block_parse(msg))
print(f"Message: 120 'x' characters")
print(f"Message length: {len(msg)} bytes")
print(f"Number of blocks: {len(blocks)}")
for i, block in enumerate(blocks):
    print(f"Block {i+1} length: {len(block)} bytes")

print("\n" + "=" * 70)

PROBLEM 3: Testing Message Padding and Parsing

--- Test 1: Empty message ---
Message: b''
Message length: 0 bytes
Number of blocks: 1
First block length: 64 bytes (expected: 64)
First block (hex): 80000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

--- Test 2: Short message 'abc' ---
Message: b'abc'
Message length: 3 bytes
Number of blocks: 1
First block length: 64 bytes (expected: 64)
First block (hex): 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018

--- Test 3: 55-byte message (boundary case) ---
Message: 55 'a' characters
Message length: 55 bytes
Number of blocks: 1
Block 1 length: 64 bytes

--- Test 4: 56-byte message (boundary case) ---
Message: 56 'a' characters
Message length: 56 bytes
Number of blocks: 2
Block 1 length: 64 bytes
Block 2 length: 64 bytes

--- Test 5: 120-byte message (multiple blocks) ---
Message: 120 'x

### Conclusion

This problem successfully implemented the SHA-256 message padding and parsing mechanism as specified in FIPS 180-4. The `block_parse()` generator function correctly pads messages of any length by appending a '1' bit (0x80), zero padding to reach the required length, and a 64-bit representation of the original message length in bits. All test cases passed, including edge cases like empty messages, boundary cases at 55 and 56 bytes, and multi-block messages. The use of a Python generator makes the implementation memory-efficient, as it yields 512-bit blocks one at a time rather than storing the entire padded message in memory. This padding mechanism is essential for SHA-256 security, preventing length extension attacks and ensuring that each unique message produces a unique padded output.

## Problem 4: Hashes

### Introduction

This problem implements the core SHA-256 hash computation function as specified in FIPS 180-4, Section 6.2.2 (page 22). This function takes a current hash value and a 512-bit message block, then produces the next hash value through 64 rounds of complex bitwise operations. Each round uses the functions implemented in Problem 1 (Ch, Maj, Σ₀, Σ₁, σ₀, σ₁), the constants generated in Problem 2 (K values), and processes blocks produced by Problem 3's padding function. The hash computation is the heart of SHA-256, transforming input data through a one-way function that makes it computationally infeasible to reverse or find collisions.

### Solution Approach

The solution implements the SHA-256 compression function which operates in several stages: First, the 512-bit message block is expanded into a 64-word message schedule using the sigma functions. Then, eight working variables (a-h) are initialized with the current hash value. For each of the 64 rounds, temporary values T1 and T2 are calculated using the Ch, Maj, Sigma0, and Sigma1 functions along with the message schedule and round constants. The working variables are updated in a specific pattern that ensures thorough mixing. Finally, the working variables are added to the current hash value to produce the next hash value. This process is repeated for each message block until all blocks are processed.

### References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Section 6.2.2, Page 22 - SHA-256 Hash Computation
- Section 5.3.3, Page 15 - Initial Hash Value
- Relevance: Official specification for the SHA-256 compression function and initial hash values.

**Wikipedia - Merkle–Damgård construction**
- URL: https://en.wikipedia.org/wiki/Merkle%E2%80%93Damg%C3%A5rd_construction
- Relevance: Explains the iterative structure used by SHA-256 where each block's hash becomes input to the next block's computation.

In [29]:
# ============================================================================
# PROBLEM 4: Hashes
# ============================================================================

# ----------------------------------------------------------------------------
# STEP 1: Define Initial Hash Value H(0)
# ----------------------------------------------------------------------------

# Initial hash values for SHA-256 (from FIPS 180-4, Section 5.3.3, Page 15)
# These are the first 32 bits of the fractional parts of the square roots
# of the first 8 prime numbers (2, 3, 5, 7, 11, 13, 17, 19)

H_INITIAL = [
    np.uint32(0x6a09e667),  # sqrt(2)
    np.uint32(0xbb67ae85),  # sqrt(3)
    np.uint32(0x3c6ef372),  # sqrt(5)
    np.uint32(0xa54ff53a),  # sqrt(7)
    np.uint32(0x510e527f),  # sqrt(11)
    np.uint32(0x9b05688c),  # sqrt(13)
    np.uint32(0x1f83d9ab),  # sqrt(17)
    np.uint32(0x5be0cd19),  # sqrt(19)
]

# K constants from Problem 2 (the 64 round constants)
# These are the first 32 bits of the fractional parts of the cube roots
# of the first 64 prime numbers
K = [
    np.uint32(0x428a2f98), np.uint32(0x71374491), np.uint32(0xb5c0fbcf), np.uint32(0xe9b5dba5),
    np.uint32(0x3956c25b), np.uint32(0x59f111f1), np.uint32(0x923f82a4), np.uint32(0xab1c5ed5),
    np.uint32(0xd807aa98), np.uint32(0x12835b01), np.uint32(0x243185be), np.uint32(0x550c7dc3),
    np.uint32(0x72be5d74), np.uint32(0x80deb1fe), np.uint32(0x9bdc06a7), np.uint32(0xc19bf174),
    np.uint32(0xe49b69c1), np.uint32(0xefbe4786), np.uint32(0x0fc19dc6), np.uint32(0x240ca1cc),
    np.uint32(0x2de92c6f), np.uint32(0x4a7484aa), np.uint32(0x5cb0a9dc), np.uint32(0x76f988da),
    np.uint32(0x983e5152), np.uint32(0xa831c66d), np.uint32(0xb00327c8), np.uint32(0xbf597fc7),
    np.uint32(0xc6e00bf3), np.uint32(0xd5a79147), np.uint32(0x06ca6351), np.uint32(0x14292967),
    np.uint32(0x27b70a85), np.uint32(0x2e1b2138), np.uint32(0x4d2c6dfc), np.uint32(0x53380d13),
    np.uint32(0x650a7354), np.uint32(0x766a0abb), np.uint32(0x81c2c92e), np.uint32(0x92722c85),
    np.uint32(0xa2bfe8a1), np.uint32(0xa81a664b), np.uint32(0xc24b8b70), np.uint32(0xc76c51a3),
    np.uint32(0xd192e819), np.uint32(0xd6990624), np.uint32(0xf40e3585), np.uint32(0x106aa070),
    np.uint32(0x19a4c116), np.uint32(0x1e376c08), np.uint32(0x2748774c), np.uint32(0x34b0bcb5),
    np.uint32(0x391c0cb3), np.uint32(0x4ed8aa4a), np.uint32(0x5b9cca4f), np.uint32(0x682e6ff3),
    np.uint32(0x748f82ee), np.uint32(0x78a5636f), np.uint32(0x84c87814), np.uint32(0x8cc70208),
    np.uint32(0x90befffa), np.uint32(0xa4506ceb), np.uint32(0xbef9a3f7), np.uint32(0xc67178f2),
]

print("=" * 70)
print("PROBLEM 4: SHA-256 Hash Computation")
print("=" * 70)
print("\nInitial Hash Values (H⁽⁰⁾):")
for i, h in enumerate(H_INITIAL):
    print(f"H[{i}] = 0x{h:08x}")

print(f"\nRound Constants (K): {len(K)} values loaded")
print("=" * 70)

PROBLEM 4: SHA-256 Hash Computation

Initial Hash Values (H⁽⁰⁾):
H[0] = 0x6a09e667
H[1] = 0xbb67ae85
H[2] = 0x3c6ef372
H[3] = 0xa54ff53a
H[4] = 0x510e527f
H[5] = 0x9b05688c
H[6] = 0x1f83d9ab
H[7] = 0x5be0cd19

Round Constants (K): 64 values loaded


In [30]:
# ----------------------------------------------------------------------------
# STEP 2: Hash Compression Function
# ----------------------------------------------------------------------------

def hash(current, block):
    """
    SHA-256 compression function - computes next hash value from current hash and message block.
    
    Implements Section 6.2.2 of FIPS 180-4 (page 22).
    
    Args:
        current: List of 8 np.uint32 values representing current hash state H(i-1)
        block: 64-byte (512-bit) message block as bytes object
    
    Returns:
        List of 8 np.uint32 values representing next hash state H(i)
    """
    
    # Step 1: Prepare the message schedule W (64 words)
    W = []
    
    # First 16 words come directly from the message block (32 bits = 4 bytes each)
    for i in range(16):
        # Extract 4 bytes and convert to 32-bit word (big-endian)
        word = np.uint32(int.from_bytes(block[i*4:(i+1)*4], byteorder='big'))
        W.append(word)
    
    # Extend to 64 words using sigma0 and sigma1 functions
    for i in range(16, 64):
        # W[t] = sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16]
        s0 = sigma0(W[i-15])
        s1 = sigma1(W[i-2])
        W.append(np.uint32(s1 + W[i-7] + s0 + W[i-16]))
    
    # Step 2: Initialize working variables with current hash value
    a, b, c, d, e, f, g, h = current
    
    # Step 3: Perform 64 rounds of hashing
    for t in range(64):
        # Calculate T1 = h + Sigma1(e) + Ch(e,f,g) + K[t] + W[t]
        T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
        
        # Calculate T2 = Sigma0(a) + Maj(a,b,c)
        T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
        
        # Update working variables
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)
    
    # Step 4: Compute the intermediate hash value H(i)
    # Add the working variables to the current hash value
    next_hash = [
        np.uint32(current[0] + a),
        np.uint32(current[1] + b),
        np.uint32(current[2] + c),
        np.uint32(current[3] + d),
        np.uint32(current[4] + e),
        np.uint32(current[5] + f),
        np.uint32(current[6] + g),
        np.uint32(current[7] + h),
    ]
    
    return next_hash


print("\n" + "=" * 70)
print("Hash compression function implemented successfully!")
print("=" * 70)


Hash compression function implemented successfully!


In [31]:
# ----------------------------------------------------------------------------
# STEP 3: Complete SHA-256 Function
# ----------------------------------------------------------------------------

def sha256(msg):
    """
    Complete SHA-256 hash function.
    
    Combines padding (Problem 3) and hash computation (Problem 4) to produce
    the final 256-bit hash of a message.
    
    Args:
        msg: bytes object containing the message to hash
    
    Returns:
        bytes: 32-byte (256-bit) SHA-256 hash digest
    """
    # Start with the initial hash value
    current_hash = H_INITIAL.copy()
    
    # Process each 512-bit block from the padded message
    for block in block_parse(msg):
        current_hash = hash(current_hash, block)
    
    # Convert the final hash value to bytes (big-endian)
    digest = b''.join(h.tobytes() for h in current_hash)
    
    # Reverse byte order for each word (numpy uses little-endian by default)
    digest = b''.join(digest[i:i+4][::-1] for i in range(0, 32, 4))
    
    return digest


print("\n" + "=" * 70)
print("Complete SHA-256 function implemented!")
print("=" * 70)


Complete SHA-256 function implemented!


###Testing SHA-256 Implementation

We test the complete SHA-256 implementation using known test vectors from FIPS 180-4 and other official sources. These tests verify that our implementation produces identical results to the standard SHA-256 algorithm

In [32]:
# ----------------------------------------------------------------------------
# STEP 4: Testing SHA-256 with Known Test Vectors
# ----------------------------------------------------------------------------

print("\n" + "=" * 70)
print("Testing SHA-256 Implementation with Known Test Vectors")
print("=" * 70)

# Test 1: Empty string
print("\n--- Test 1: Empty string ---")
msg = b""
result = sha256(msg)
expected = "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"
print(f"Message: {repr(msg)}")
print(f"Calculated: {result.hex()}")
print(f"Expected:   {expected}")
print(f"Match: {'✓ PASS' if result.hex() == expected else '✗ FAIL'}")

# Test 2: "abc"
print("\n--- Test 2: 'abc' ---")
msg = b"abc"
result = sha256(msg)
expected = "ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad"
print(f"Message: {repr(msg)}")
print(f"Calculated: {result.hex()}")
print(f"Expected:   {expected}")
print(f"Match: {'✓ PASS' if result.hex() == expected else '✗ FAIL'}")

# Test 3: "abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq"
print("\n--- Test 3: Long alphabetic string ---")
msg = b"abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq"
result = sha256(msg)
expected = "248d6a61d20638b8e5c026930c3e6039a33ce45964ff2167f6ecedd419db06c1"
print(f"Message: {repr(msg)}")
print(f"Calculated: {result.hex()}")
print(f"Expected:   {expected}")
print(f"Match: {'✓ PASS' if result.hex() == expected else '✗ FAIL'}")

# Test 4: "The quick brown fox jumps over the lazy dog"
print("\n--- Test 4: 'The quick brown fox...' ---")
msg = b"The quick brown fox jumps over the lazy dog"
result = sha256(msg)
expected = "d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592"
print(f"Message: {repr(msg)}")
print(f"Calculated: {result.hex()}")
print(f"Expected:   {expected}")
print(f"Match: {'✓ PASS' if result.hex() == expected else '✗ FAIL'}")

# Test 5: Single character "a"
print("\n--- Test 5: Single character 'a' ---")
msg = b"a"
result = sha256(msg)
expected = "ca978112ca1bbdcafac231b39a23dc4da786eff8147c4e72b9807785afee48bb"
print(f"Message: {repr(msg)}")
print(f"Calculated: {result.hex()}")
print(f"Expected:   {expected}")
print(f"Match: {'✓ PASS' if result.hex() == expected else '✗ FAIL'}")

print("\n" + "=" * 70)
print("SHA-256 Implementation Testing Complete!")
print("=" * 70)


Testing SHA-256 Implementation with Known Test Vectors

--- Test 1: Empty string ---
Message: b''
Calculated: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Expected:   e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Match: ✓ PASS

--- Test 2: 'abc' ---
Message: b'abc'
Calculated: ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Expected:   ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad
Match: ✓ PASS

--- Test 3: Long alphabetic string ---
Message: b'abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq'
Calculated: 248d6a61d20638b8e5c026930c3e6039a33ce45964ff2167f6ecedd419db06c1
Expected:   248d6a61d20638b8e5c026930c3e6039a33ce45964ff2167f6ecedd419db06c1
Match: ✓ PASS

--- Test 4: 'The quick brown fox...' ---
Message: b'The quick brown fox jumps over the lazy dog'
Calculated: d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e592
Expected:   d7a8fbb307d7809469ca9abcb0082e4f8d5651e46d3cdb762d02d0bf37c9e5

/tmp/ipykernel_2067/1849051005.py:33: RuntimeWarning: overflow encountered in scalar add
  W.append(np.uint32(s1 + W[i-7] + s0 + W[i-16]))
/tmp/ipykernel_2067/1849051005.py:41: RuntimeWarning: overflow encountered in scalar add
  T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + K[t] + W[t])
/tmp/ipykernel_2067/1849051005.py:44: RuntimeWarning: overflow encountered in scalar add
  T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
/tmp/ipykernel_2067/1849051005.py:50: RuntimeWarning: overflow encountered in scalar add
  e = np.uint32(d + T1)
/tmp/ipykernel_2067/1849051005.py:54: RuntimeWarning: overflow encountered in scalar add
  a = np.uint32(T1 + T2)
/tmp/ipykernel_2067/1849051005.py:60: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[1] + b),
/tmp/ipykernel_2067/1849051005.py:62: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[3] + d),
/tmp/ipykernel_2067/1849051005.py:63: RuntimeWarning: overflow encountered in scalar add
  np.uint32(current[4] + e),

### Conclusion
Problem 4 brings everything together into a SHA-256 implementation. The hash compression function takes the bitwise operations we built in problem 1 , applies the constants we calculated in problem 2 , and processes the padded message blocks from problem 3 . Each message block goes through 64 rounds of mixing using temporary variables and the working variables a through h, with each round thoroughly scrambling the data. We tested the implementaion against five official test vectors, and every single one matched perfectly with empty strings, shorts messages and longer inputs all produced the correct SHA-256 hashes. The overflow warnings that appear are actually normal for SHA-256 since its deliberately uses modul 2*3*2 arithmitic , meaning numbers wrap around at 32 bits by design. With a fully working SHA-256 implementation verified against standard test cases, we can now use it to crack password hashes in Problem 5

## Problem 5: Passwords

### Introduction

This problem demonstrates the vulnerability of unsalted password hashes through a practical password cracking exercise. Given three SHA-256 hashes of common passwords, the task is to determine the original passwords and explain the attack methodology. While SHA-256 is cryptographically secure as a hash function, using it alone for password storage is insufficient because common passwords can be cracked through dictionary attacks, rainbow tables, or brute force attempts. This problem illustrates why real-world password storage requires additional protections like salting (adding random data before hashing) and key derivation functions (like bcrypt, scrypt, or Argon2) that are intentionally slow to compute.

### Solution Approach

The solution uses a dictionary attack approach by testing common passwords against the given hashes. Since the passwords are described as "common," they likely appear in standard password lists or dictionaries. For each candidate password, we encode it as UTF-8 bytes, compute its SHA-256 hash using our implementation from Problem 4, and compare against the target hashes. This demonstrates how attackers can efficiently crack weak passwords even when hashed with strong algorithms. After identifying the passwords, we analyze why this attack succeeded and propose proper password hashing techniques including  using purpose-built password hashing functions with adjustable work factors.

### References

**FIPS 180-4: Secure Hash Standard**
- National Institute of Standards and Technology (NIST)
- URL: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
- Relevance: Specification for SHA-256 used to hash the passwords.

**OWASP Password Storage Cheat Sheet**
- URL: https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html
- Relevance: Industry best practices for secure password storage, including salting, hashing, and key derivation functions.

**Wikipedia - Salt (cryptography)**
- URL: https://en.wikipedia.org/wiki/Salt_(cryptography)
- Relevance: Explains how adding random salt prevents rainbow table attacks and makes each password hash unique.

**Wikipedia - Key derivation function**
- URL: https://en.wikipedia.org/wiki/Key_derivation_function
- Relevance: Describes purpose-built functions like bcrypt and Argon2 designed specifically for password hashing with adjustable computational cost.